In [1]:
from datasets import load_dataset
from transformers import BertTokenizer
from datasets import DatasetDict
import numpy as np
# Instantiate a tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForMaskedLM

In [2]:
train_dataset = load_dataset("csv", data_files="/home/ghan/caml-mimic-fixed-/mimicdata/mimic3/train_50.csv")
eval_dataset = load_dataset("csv", data_files="/home/ghan/caml-mimic-fixed-/mimicdata/mimic3/dev_50.csv")
predict_dataset = load_dataset("csv", data_files="/home/ghan/caml-mimic-fixed-/mimicdata/mimic3/test_50.csv")

Using custom data configuration default-2828371e2f74ebe3
Found cached dataset csv (/home/ghan/.cache/huggingface/datasets/csv/default-2828371e2f74ebe3/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-6249d4195201014c
Found cached dataset csv (/home/ghan/.cache/huggingface/datasets/csv/default-6249d4195201014c/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-b097882698f56793
Found cached dataset csv (/home/ghan/.cache/huggingface/datasets/csv/default-b097882698f56793/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
train_dataset['train']

Dataset({
    features: ['SUBJECT_ID', 'HADM_ID', 'TEXT', 'LABELS', 'length'],
    num_rows: 8066
})

In [4]:
def encode(examples):
    encoded = tokenizer(examples['TEXT'])
    examples['length'] = len(encoded['input_ids'])
    return examples

In [5]:
predict_dataset = predict_dataset.map(encode)

Loading cached processed dataset at /home/ghan/.cache/huggingface/datasets/csv/default-b097882698f56793/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-333be23f5f1ef6a4.arrow


In [6]:
eval_dataset = eval_dataset.map(encode)

Loading cached processed dataset at /home/ghan/.cache/huggingface/datasets/csv/default-6249d4195201014c/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-f24d9ee8898af698.arrow


In [7]:
train_dataset = train_dataset.map(encode)

Loading cached processed dataset at /home/ghan/.cache/huggingface/datasets/csv/default-2828371e2f74ebe3/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-96fb314c6d4d2df1.arrow


In [8]:
predict_dataset['train']['length']

[268,
 506,
 516,
 541,
 545,
 666,
 645,
 610,
 631,
 625,
 643,
 629,
 673,
 666,
 662,
 667,
 703,
 729,
 710,
 794,
 812,
 792,
 663,
 906,
 752,
 731,
 788,
 824,
 818,
 825,
 887,
 933,
 867,
 875,
 911,
 889,
 943,
 896,
 812,
 924,
 966,
 914,
 892,
 979,
 993,
 964,
 885,
 1005,
 908,
 909,
 920,
 979,
 1075,
 1043,
 1059,
 983,
 1071,
 990,
 1068,
 1037,
 1018,
 986,
 1012,
 1009,
 1040,
 1051,
 1127,
 1178,
 1137,
 1061,
 1126,
 1063,
 1232,
 1062,
 1185,
 1156,
 1096,
 1099,
 1017,
 1079,
 1108,
 1113,
 1143,
 1152,
 1144,
 1197,
 1078,
 1136,
 1135,
 1082,
 1130,
 1234,
 1131,
 1207,
 1228,
 1202,
 1088,
 1143,
 1151,
 1278,
 1222,
 1151,
 1177,
 1225,
 1184,
 1091,
 1303,
 1224,
 1125,
 1154,
 1152,
 1201,
 1192,
 1207,
 1150,
 1240,
 1275,
 1263,
 1256,
 1176,
 1185,
 1251,
 1381,
 1331,
 1148,
 1349,
 1328,
 1130,
 1216,
 1204,
 1267,
 1265,
 1317,
 1274,
 1305,
 1305,
 1260,
 1200,
 1193,
 1262,
 1197,
 1297,
 1235,
 1204,
 1282,
 1259,
 1436,
 1264,
 1221,
 1370,
 133

In [9]:
lengths=train_dataset['train']['length']+predict_dataset['train']['length']+eval_dataset['train']['length']

In [10]:
predict_lengths=predict_dataset['train']['length']

In [11]:
len(lengths)

11368

In [18]:
quartiles = np.percentile(lengths, [25, 50, 75], axis=0)

In [20]:
quartiles

array([1432.  , 2022.  , 2741.25])

In [21]:
def add_length_feature(examples):

    length=examples['length']
    if length <= quartiles[0]:
        length_feature=1
    elif length <= quartiles[1]:
        length_feature=2
    elif length <= quartiles[2]:
        length_feature=3
    else:
        length_feature=4
    examples['length_feature'] = length_feature
    return examples

train_dataset = train_dataset['train'].map(add_length_feature)
predict_dataset = predict_dataset['train'].map(add_length_feature)
eval_dataset = eval_dataset['train'].map(add_length_feature)

  0%|          | 0/8066 [00:00<?, ?ex/s]

  0%|          | 0/1729 [00:00<?, ?ex/s]

  0%|          | 0/1573 [00:00<?, ?ex/s]

In [22]:
train_dataset

Dataset({
    features: ['SUBJECT_ID', 'HADM_ID', 'TEXT', 'LABELS', 'length', 'length_feature'],
    num_rows: 8066
})

In [23]:
train_df = train_dataset.to_pandas()
eval_df=eval_dataset.to_pandas()
predict_df=predict_dataset.to_pandas()

In [24]:
train_df.to_json("/home/ghan/datasets/mimic_train50.json", orient="records", lines=True)
eval_df.to_json("/home/ghan/datasets/mimic_dev50.json", orient="records", lines=True)
predict_df.to_json("/home/ghan/datasets/mimic_test50.json", orient="records", lines=True)